In [22]:
from cmath import cos
from inspect import _void
import math
from sys import flags
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal, stats
import scipy.signal as spsg
import os
from sklearn.cluster import KMeans
from scipy.spatial import distance
import time
import mat73
import os
from os.path import join
TMAX = 111 # Length of time series
NREGIONS = 82
basePath = "tDCS_timeseries"
awakePath = join(basePath, "timeseries_CoCoMac_tDCS_awake")
anesthPath = join(basePath, "timeseries_CoCoMac_tDCS_anesthesia")


In [ ]:
# Angular difference function
def adif(a, b):
    return 2*math.pi-abs(a-b) if abs(a-b)>math.pi else abs(a-b)
vec_adif = np.vectorize(adif,otypes=[np.ndarray])
def map_adif(t0xn):
    """ Takes a timepoint of all the nodes and generates the phase differenes for all of them
    Parameters
    ----------
    t0xn: np.array(1,N)
        One timesample of an original nodesxsteps timeseries
    Returns
    ----------
    np.array(N,N)
        NxN matrix of phase differences
    """
    return np.array(list(map(lambda node: vec_adif(t0xn,node),t0xn)))
def create_phase_dif(ts):
    """ Takes a whole trial and generates the phases differences for all timepoints
    Parameters
    ----------
    t0xn: np.array(N,T)
        A trial of T steps registiring N channels or nodes
    Returns
    ----------
    np.array(N,N)
        NxNxT matrix of phase differences
    """
    return np.apply_along_axis(map_adif,-1,ts)

vectorized_phase_diffs = np.apply_along_axis(create_phase_dif,0,instantaneous_phase)

In [21]:
awakeFnames = os.listdir(awakePath)
anesthFnames = os.listdir(anesthPath)
ts_all = np.zeros((len(awakeFnames)+len(anesthFnames),NREGIONS,TMAX))
for i,fn in enumerate(os.listdir(join(awakePath))):
    tmp = loadmat(join(awakePath,fn))
    ts_all[i, :, :] = np.swapaxes(tmp['scans'][:TMAX,:],0,1) 

for i,fn in enumerate(os.listdir(join(anesthPath))):
    tmp = loadmat(join(anesthPath,fn))
    ts_all[i+len(awakeFnames), :, :] = np.swapaxes(tmp['scans'][:TMAX,:],0,1) 
ts_all.shape

(554, 82, 111)

In [3]:
#Data Anesthesia DBS
data_ts = loadmat('data_ts_DBS_CoCoMac_0.0025-0.05_Cleaned_Camilo_version.mat')
ts_aw=np.squeeze(data_ts['ts_aw'])  #36 scans
ts_off=np.squeeze(data_ts['ts_off']) #28 scans
ts_on_3V=np.squeeze(data_ts['ts_on_3V'])  #31 scans
ts_on_3V_control=np.squeeze(data_ts['ts_on_3V_control']) #18 scans
ts_on_5V=np.squeeze(data_ts['ts_on_5V'])  #25 scans
ts_on_5V_control=np.squeeze(data_ts['ts_on_5V_control']) #18 scans
ts_all = np.concatenate((ts_aw,ts_off,ts_on_3V,ts_on_3V_control,ts_on_5V,ts_on_5V_control),axis=0)

In [4]:
np.info(ts_all[0])

class:  ndarray
shape:  (82, 500)
strides:  (8, 656)
itemsize:  8
aligned:  True
contiguous:  False
fortran:  True
data pointer: 0x55ffa1c6efe0
byteorder:  little
byteswap:  False
type: float64


In [23]:
# Z-score
ts_all_zscored = np.zeros((len(ts_all),NREGIONS,TMAX)) # Storage for z-scored filtered signals
for i in range(len(ts_all)):
    ts_all_zscored[i]=np.array(stats.zscore(ts_all[i],axis=1))

In [24]:
# Detrend signal
for i in range(len(ts_all_zscored)):
    ts_all_zscored[i] = spsg.detrend(ts_all_zscored[i],type='linear')
# Demeaning signal
for i in range(len(ts_all_zscored)):
    ts_all_zscored[i] = ts_all_zscored[i]-np.mean(ts_all_zscored[i])
signal = np.array(ts_all_zscored)

In [25]:
# Reverse the signal, and Concatenation from both sides.
signal_rev = np.flip(signal, axis=2)
signal_rev_concat = np.concatenate((signal_rev, signal, signal_rev), axis=2)


# Butterworth filter parameters
n_order = 2
TR = 2.4 # in seconds
fnq = 1/(2*TR) # Nyquist frequency
low_f = 0.0025 # lower cutoff for bandpass filter
high_f = 0.05 # upper cutoff for bandpass filter
Wn = [low_f/fnq,high_f/fnq]
b,a = spsg.iirfilter(n_order,Wn,btype='bandpass',ftype='butter', output='ba')
# Apply Butterworth filter
filt_ts = np.empty_like(signal_rev_concat)
for i in range(len(ts_all_zscored)):
    filt_ts = spsg.filtfilt(b,a,signal_rev_concat,axis=1, padtype='odd', padlen=3*(max(len(b),len(a))-1))

In [26]:
# Now we can cut the signal back to its original size by taking the middle part
signal_filtered = filt_ts[:,:,len(signal_rev[0][0]):len(signal_rev[0][0])+len(signal[0][0])]

# And we can demean it again
signal_filtered = signal_filtered - np.mean(signal_filtered, axis=2, keepdims=True)


# Analytical continuation of our signal
analytic_signal = spsg.hilbert(signal_filtered)
amplitude_envelope = np.abs(analytic_signal)
instantaneous_phase = np.unwrap(np.angle(analytic_signal)) # Unwrap doesn't do anything?
# signal_reconstr = amplitude_envelope*np.cos(instantaneous_phase) # To ensure that we can reconstruct our initial signal from the hilbert transform

In [1]:
np.info(instantaneous_phase)

NameError: name 'np' is not defined

In [43]:
len(instantaneous_phase[0][0])

111

In [28]:
# Now we can calculate the phase difference between each pair of regions using the adif function
# We will store the phase differences in a matrix of size (n_regions, n_regions, n_timepoints)
phase_diff = np.zeros((len(instantaneous_phase),len(instantaneous_phase[0]),len(instantaneous_phase[0]),len(instantaneous_phase[0][0])))
for ind in range(len(instantaneous_phase)):
    for t in range(len(instantaneous_phase[0][0])):
        for i in range(len(instantaneous_phase[0])):
            for j in range(len(instantaneous_phase[0])):
                if i==j:
                    phase_diff[ind,i,j,t] = 0
                else:
                    phase_diff[ind,i,j,t] = np.cos(adif(instantaneous_phase[ind][i][t], instantaneous_phase[ind][j][t]))

# Quite lengthy operation: hate those for loops :'(
# Find a way through numpy array handling

In [ ]:
len(instantaneous_phase[0])


6724

In [37]:
len(instantaneous_phase[0][0])

111

In [40]:
len(instantaneous_phase)

554

In [ ]:
# y = int(len(instantaneous_phase)*len(instantaneous_phase[0][0][0]))
# x = int(len(instantaneous_phase[0][0])/2*(len(instantaneous_phase[0][0])-1))
# Vectorize the superior triangular
vectorized = np.zeros((3321,TMAX*len(instantaneous_phase)))
for ind in range(len(instantaneous_phase)):
    for t in range(len(instantaneous_phase[0][0])):
        temp = np.array(())
        for j in range(len(instantaneous_phase[0])):
            for i in range(len(instantaneous_phase[0])):
                if i>j:
                    temp = np.append(temp,phase_diff[ind,i,j,t])        
        vectorized[:,ind*TMAX+t] = temp
        

vectorized = vectorized.T
# Quite lengthy operation: hate those for loops :'(
# Maybe a way through numpy array handling
# Could probably actually do both the i>j and the cos(sigma_phi) at the same time...

In [ ]:
# TSCM K-Means
# AW+OFF+ON5V
    
_5V_vector = np.concatenate((vectorized[:,0:32000], vectorized[:,56500:69000]), axis=1)

    
labels_5V = ()
centroids_5V = ()

k=7
kmeans_5V = KMeans(n_clusters=k, random_state=1, n_init=500,init='random').fit(_5V_vector)
labels_5V = kmeans_5V.labels_
centroids_selv_pp = kmeans_5V.cluster_centers_
np.save('TSCM_DBS_centroids_5V k='+ str(k), centroids_5V)    
np.save('TSCM_DBS_labels_5V k='+ str(k), labels_5V)

In [13]:
# Concatenate only awake, lpp and dpp for k-means
# TSCM K-Means
# AW+OFF+ON5VC
    
_5VC_vector = np.concatenate((vectorized[:,0:32000], vectorized[:,69000:78000]), axis=1)

    
labels_5VC = ()
centroids_5VC = ()

k=7
kmeans_5VC = KMeans(n_clusters=k, random_state=1, n_init=500,init='random').fit(_5VC_vector)
labels_5VC = kmeans_5VC.labels_
centroids_selv_pp = kmeans_5VC.cluster_centers_
np.save('TSCM_DBS_centroids_5VC k='+ str(k), centroids_5VC)    
np.save('TSCM_DBS_labels_5VC k='+ str(k), labels_5VC)

In [14]:
Time_per_rec = len(ts_all[0][0])
N=len(ts_all[0])
Total_patient = len(ts_all)
Leading_Eig = np.empty((Total_patient*Time_per_rec,N))
for ind in range(Total_patient):
    for t in range(Time_per_rec):
            eigenvalues, eigenvectors = np.linalg.eig(phase_diff[ind,:,:,t])
            
            V1 = eigenvectors[:, np.argmax(eigenvalues)]
            
            mean_positive = np.mean(V1 > 0)

            if mean_positive > 0.5:
                V1 = -V1
            elif mean_positive == 0.5 and np.sum(V1[V1 > 0]) > -np.sum(V1[V1 < 0]):
                V1 = -V1
            Leading_Eig[Time_per_rec*ind+t] = V1

/opt/miniconda/envs/jupyterhub/lib/python3.6/site-packages/ipykernel_launcher.py:17: ComplexWarning: Casting complex values to real discards the imaginary part


In [15]:
_5V_vec = np.concatenate((Leading_Eig[:,0:32000], Leading_Eig[:,56500:69000]), axis=1)

# K-Means 5V
                                          
labels_aw_pp = ()
centroids_aw_pp = ()

k=7
kmeans_5V = KMeans(n_clusters=k, random_state=1, n_init=500,init='random').fit(_5V_vec)
labels_5V = kmeans_5V.labels_
centroids_5V = kmeans_5V.cluster_centers_
np.save('centroids_DBS_LEiDA_5V k='+ str(k), centroids_5V)    
np.save('labels_DBS_LEiDA_5V k='+ str(k), labels_5V)

In [16]:
_5VC_vec = np.concatenate((Leading_Eig[:,0:32000], Leading_Eig[:,69000:78000]), axis=1)

# K-Means 5VC
                                          
labels_aw_pp = ()
centroids_aw_pp = ()

k=7
kmeans_5VC = KMeans(n_clusters=k, random_state=1, n_init=500,init='random').fit(_5VC_vec)
labels_5VC = kmeans_5VC.labels_
centroids_5VC = kmeans_5VC.cluster_centers_
np.save('centroids_DBS_LEiDA_5VC k='+ str(k), centroids_5VC)    
np.save('labels_DBS_LEiDA_5VC k='+ str(k), labels_5VC)

In [40]:
len(labels_5VC)

82

In [41]:
np.info(_5VC_vec)

class:  ndarray
shape:  (78000, 82)
strides:  (656, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x7f44405b5010
byteorder:  little
byteswap:  False
type: float64
